In [1]:
from basic.utils.run.train import train
from basic.utils.common_utils import put_data_to_gpu
from basic.utils.run.train import Trainer
from basic.utils.vis_utils import VisualWindow
import easydict
import yaml
import torch
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

/home/ph/ENV/anaconda3/lib/python3.8/site-packages/setuptools/distutils_patch.py:25: UserWarning: Distutils was imported before Setuptools. This usage is discouraged and may exhibit undesirable behaviors or errors. Please use Setuptools' objects directly or at least import Setuptools first.
  warnings.warn(


In [2]:
train_cfg_path = '../kitti/cfg/train_cfg.yaml'
with open(train_cfg_path, 'rb') as f:
    train_cfg = easydict.EasyDict(yaml.load(f))
train_cfg

<ipython-input-2-b733ea9d0706>:3: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  train_cfg = easydict.EasyDict(yaml.load(f))


{'DATASET_CONFIG': '/home/ph/Desktop/PointCloud/utils_my/kitti/cfg/kitti_dataset.yaml',
 'CLASS_NAMES': ['Car'],
 'BATCH': 6,
 'EPOCHS': 50,
 'MODEL': {'NAME': 'SECOND',
  'CONFIG': '/home/ph/Desktop/PointCloud/utils_my/basic/model/model_cfg/second.yaml'},
 'OPTIMIZER': {'NAME': 'Adam', 'LR': 6e-05}}

In [3]:
# train(train_cfg)

In [4]:
from basic.model.second import SECOND
from pathlib import Path
from kitti.kitti_dataset import get_dataloader

cfg_path = Path("../kitti/cfg/kitti_dataset.yaml")
dataloader = get_dataloader(data_cfg_path=cfg_path, class_name_list=['Car'], batch_size=2, training=True)
for data in dataloader:
    test_data = data
    break
test_data = put_data_to_gpu(test_data)
test_data

/home/ph/Desktop/PointCloud/utils_my/kitti/kitti_dataset.py:30: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  cfg = EasyDict(yaml.load(f))


{'frame_id': array(['006234', '006277'], dtype='<U6'),
 'gt_boxes': tensor([[[ 14.9991,  -2.7765,  -0.9938,   3.8287,   1.4718,   1.3949,   2.7272,
             1.0000],
          [ 32.5352, -18.7313,  -0.7581,   3.5113,   1.5488,   1.3372,  -1.2060,
             1.0000],
          [ 29.6153, -40.8371,  -1.8965,   2.8763,   1.7316,   1.5103,  -1.8960,
             1.0000],
          [ 29.9629,  -3.8575,  -0.8332,   4.0981,   1.6739,   1.4141,   1.2272,
             1.0000],
          [ 30.9776,  -8.8905,  -0.4778,   4.3193,   1.6065,   1.5969,   2.7572,
             1.0000],
          [ 19.2035, -10.8810,  -0.7083,   3.8864,   1.6354,   1.2698,  -0.3960,
             1.0000],
          [ 23.1628, -14.6068,  -0.5840,   3.9345,   1.5392,   1.3564,  -1.2760,
             1.0000],
          [ 34.1524,  -5.2083,  -0.8413,   3.7902,   1.5296,   1.3564,   1.2372,
             1.0000],
          [ 19.7918, -21.8636,  -0.4986,   4.2616,   1.4622,   1.4141,  -2.3160,
             1.0000],
      

In [5]:
%matplotlib auto
from basic.utils.run.train import Predictor
model_path = '/home/ph/Desktop/PointCloud/utils_my/basic/utils/run/check_point/2021-11-04::19:09:51/epoch19.pkl'
predictor = Predictor(model_path, class_names=['Car'])
# predictor = Predictor(model_path, class_names=['Car'], data_cfg_path=cfg_path, batch_size=2)
# predictor.predict_and_visualize()
preds_dict = predictor.predict_bbox(test_data, eval=True)
preds_dict[0]

Using matplotlib backend: Qt5Agg


(tensor([[0.2485, 0.7515],
         [0.6901, 0.3099],
         [0.6915, 0.3085],
         [0.7164, 0.2836],
         [0.7264, 0.2736],
         [0.7382, 0.2618],
         [0.7439, 0.2561],
         [0.7463, 0.2537],
         [0.7514, 0.2486],
         [0.7522, 0.2478],
         [0.7524, 0.2476],
         [0.7527, 0.2473],
         [0.7532, 0.2468],
         [0.7532, 0.2468],
         [0.7534, 0.2466],
         [0.7534, 0.2466],
         [0.7534, 0.2466],
         [0.7535, 0.2465],
         [0.7536, 0.2464],
         [0.7538, 0.2462],
         [0.7540, 0.2460],
         [0.7540, 0.2460],
         [0.7541, 0.2459],
         [0.7541, 0.2459],
         [0.7542, 0.2458],
         [0.7543, 0.2457],
         [0.7543, 0.2457],
         [0.7544, 0.2456],
         [0.7544, 0.2456],
         [0.7544, 0.2456],
         [0.7545, 0.2455],
         [0.7546, 0.2454],
         [0.7546, 0.2454],
         [0.7546, 0.2454],
         [0.7547, 0.2453],
         [0.7547, 0.2453],
         [0.7547, 0.2453],
 

In [6]:
preds_dict[1]

(tensor([[0.2485, 0.7515],
         [0.6570, 0.3430],
         [0.6835, 0.3165],
         [0.6844, 0.3156],
         [0.6943, 0.3057],
         [0.6983, 0.3017],
         [0.6984, 0.3016],
         [0.7028, 0.2972],
         [0.7032, 0.2968],
         [0.7047, 0.2953],
         [0.7056, 0.2944],
         [0.7063, 0.2937],
         [0.7064, 0.2936],
         [0.7076, 0.2924],
         [0.7079, 0.2921],
         [0.7092, 0.2908],
         [0.7093, 0.2907],
         [0.7103, 0.2897],
         [0.7120, 0.2880],
         [0.7130, 0.2870],
         [0.7132, 0.2868],
         [0.7139, 0.2861],
         [0.7145, 0.2855],
         [0.7151, 0.2849],
         [0.7161, 0.2839],
         [0.7163, 0.2837],
         [0.7184, 0.2816],
         [0.7190, 0.2810],
         [0.7199, 0.2801],
         [0.7204, 0.2796],
         [0.7205, 0.2795],
         [0.7213, 0.2787],
         [0.7217, 0.2784],
         [0.7224, 0.2776],
         [0.7227, 0.2773],
         [0.7235, 0.2765],
         [0.7237, 0.2763],
 

In [7]:
from kitti.io.kitti_io import KittiIo
file_reader = KittiIo(root="/home/ph/Dataset/KITTI/training")
for i in range(2):
    bbox = preds_dict[i][1][0:5]
    mask = test_data['points'][:, 0] == i
    w = VisualWindow(mode='3d', file_reader=file_reader)
    w.draw_point_cloud(test_data['points'][mask][:, 1:].cpu())
    w.draw_boxes3d(test_data['gt_boxes'][i].cpu().numpy(), 'corner')
    w.draw_boxes3d(bbox.cpu().detach().numpy(), 'corner', c='r')

In [8]:
from kitti.io.kitti_io import KittiIo
file_reader = KittiIo(root="/home/ph/Dataset/KITTI/training")
# pos = preds_dict['pred_bbox'].cpu().numpy()[preds_dict['pred_bbox_labels'] != 0]
if preds_dict['frame_inds'] is not None:
    for i in preds_dict['frame_inds'].unique():
        mask = test_data['points'][:, 0] == i
        w = VisualWindow(mode='3d', file_reader=file_reader)
        w.draw_point_cloud(test_data['points'][mask][:, 1:].cpu())
        w.draw_boxes3d(test_data['gt_boxes'][i].cpu().numpy(), 'corner')
        w.draw_boxes3d(preds_dict['pred_bbox'].cpu().numpy(), 'corner', c='r')

TypeError: list indices must be integers or slices, not str

In [ ]:
predictor.model

In [ ]:

s = torch.randn(10, 1000, 4)
max_s,arg_s = s.max(dim=-1)
_,topk = max_s.topk(5)
max_s.shape

In [ ]:
arg_s

In [ ]:
topk_labels = torch.stack([arg_s[i, topk[i]] for i in range(10)], dim=0)

In [ ]:
topk[0][topk_labels[0] > 0]

In [ ]:
data = torch.arange(1024).reshape(2, 4, 8, 16)
torch.where(data==200)

In [ ]:
data1 = data.view(-1, 1)
data1[200]

In [ ]:
data2 = data.permute(0,2,1,3).contiguous()
torch.where(data2==200)

In [ ]:
data3 = data2.permute(0, 2, 1, 3).contiguous()
data3.view(-1, 1)[200]

In [ ]:
data3.reshape(-1, 1)[200]

In [ ]:
d = data.view(2, 4, 4, 2, 16)

In [ ]:
torch.where(d==200)

In [ ]:
d2 = data.view(2, 4, 2, 4, 16)
torch.where(d2==200)

In [ ]:
        # cls_pred = cls_pred.view(B, A, cls_pred.size(1) //A, H, W, -1).contiguous()  # B, C, H, W, A
        # reg_pred = reg_pred.view(B, A, reg_pred.size(1) //A, H, W, -1).contiguous()  # B, 7, H, W, A
        # # B, C, H, W, A -> B, W, H, A, C -> B, H*W*A, C
        # cls_pred = cls_pred.permute(0, 4, 3, 1, 2).contiguous().view(B, H * W * A, -1)
        # reg_pred = reg_pred.permute(0, 4, 3, 1, 2).contiguous().view(B, H * W * A, -1)